In [1]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np
from scipy.spatial import distance

In [2]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [3]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [4]:
def nearestClass(neighbors):
    classVote = {}
    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1
    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [5]:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)

In [6]:
directory = "C:\\Users\\User\\Desktop\\SEM VI\\Ml\\genres\\"
f= open("my.dat" ,'wb')
i=0
for folder in os.listdir(directory):
    i+=1
    if i == 11:
        break
    for file in os.listdir(directory+folder):
        (rate,sig) = wav.read(directory+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)
f.close()

In [7]:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  

    for x in range(len(dataset)):
        if random.random() <split :      
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])  

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)

In [8]:
print(testSet)

[(array([ 70.00291522,   6.96591781, -18.71681431,   8.84196105,
        -8.01016039,   3.43449211, -31.36002343,   1.69970775,
       -18.67915656,  -6.93499514, -12.81097953,  -4.71106351,
        -6.55271779]), array([[ 9.57584757e+01, -5.65310912e+01, -5.90639441e+01,
        -1.45119260e+01, -6.14077950e+01, -3.42481308e+01,
        -4.66116113e+00,  2.49012307e+01, -2.48550402e+01,
         6.20214792e+01,  1.01464095e+01, -2.24626140e+01,
         2.00679871e+01],
       [-5.65310912e+01,  1.32572970e+02,  2.36316291e+00,
        -5.02681131e+01,  9.68021395e+01, -4.55083237e+00,
         9.45436387e+00, -5.57817894e+01,  2.20355677e+01,
        -9.88945983e+01,  2.21041720e+01,  1.79191596e+01,
        -1.03398875e+00],
       [-5.90639441e+01,  2.36316291e+00,  9.52344968e+01,
         2.83875850e+01, -9.98877813e-01,  3.38520740e+01,
         4.41553380e+00,  2.03612486e+01,  8.00864331e+00,
        -1.92705517e+01, -2.25526020e+01,  6.22419666e+00,
        -1.57710664e+01],


In [9]:
leng = len(testSet)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 11))) 

accuracy1 = getAccuracy(testSet , predictions)
print(predictions)


[3, 6, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 3, 1, 1, 1, 1, 1, 10, 1, 10, 5, 4, 1, 1, 1, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 9, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 9, 3, 3, 3, 5, 3, 3, 9, 3, 4, 10, 3, 10, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 4, 4, 3, 4, 4, 8, 4, 9, 4, 4, 4, 4, 4, 9, 10, 4, 4, 4, 10, 4, 4, 4, 9, 5, 5, 4, 4, 5, 10, 5, 4, 5, 5, 8, 4, 5, 4, 5, 5, 5, 5, 9, 5, 8, 8, 9, 5, 5, 5, 5, 8, 9, 5, 5, 5, 9, 5, 2, 3, 9, 3, 1, 6, 6, 6, 2, 6, 1, 4, 6, 8, 4, 10, 2, 4, 4, 6, 3, 6, 7, 6, 6, 6, 6, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7, 7, 4, 7, 7, 5, 7, 7, 4, 7, 7, 7, 7, 10, 7, 8, 8, 8, 8, 3, 3, 5, 8, 4, 8, 4, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 4, 9, 9, 9, 9, 9, 9, 5, 9, 9, 4, 9, 5, 5, 9, 9, 9, 9, 8, 5, 5, 5, 9, 9, 9, 8, 4, 9, 1, 10, 10, 9, 10, 4, 10, 4, 4, 4, 10, 10, 4, 10, 4, 4, 4, 3, 4, 9, 10, 4, 10, 4, 10, 4, 10, 10, 10, 10, 7]


In [10]:
import matplotlib.pyplot as plt
plt.plot(list(range(5,50,2)),result)
plt.title(' Variation - acc/k ')
plt.xlabel('K')
plt.ylabel('Accuracy')
plt.show()

NameError: name 'result' is not defined

In [ ]:
max(result)

In [ ]:
y_t = []
for x in range (len(testSet)):
    y_t.append(testSet[x][-1])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
cf_matrix = confusion_matrix(y_t,predictions)
print(cf_matrix)
print(classification_report(y_t,predictions))